# James' Version of Scraping.ipynb following Jeff's Workflow

**BOOKMARKS/TABLE OF CONTENTS:**

[Bookmark: 12/23/19](#📓BOOKMARK-FOR-JAMES-&-JEFF)

### To Do:
- Get links from `custom-badges`
    - Use Jeff's code to scrapen the lesson completion from learn
    - Add Portfolio project page extraction (also in custom-badges)

___

In [1]:
!pip install -U fsds_100719
from fsds_100719.imports import *

fsds_1007219  v0.5.13 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds_100719,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


In [2]:
import requests
import time

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
driver = webdriver.Chrome()

In [6]:
url = 'https://instruction.learn.co/staff/students'

In [7]:
driver.get(url)

## Username for Github

In [8]:
with open("/Users/jamesirving/.secret/learn_login.json") as f:
    import json
    login_info = json.loads(f.read())
login_info.keys()

dict_keys(['username', 'password'])

In [9]:
username = driver.find_element_by_xpath('//*[@id="login_field"]')
username.send_keys(login_info['username'])

## Password for Github

In [10]:
password = driver.find_element_by_xpath('//*[@id="password"]')
password.send_keys(login_info['password'])

In [11]:
sign_in = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sign_in.click()

In [12]:
# click view students to get to the students section
view = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div[1]/div/div[2]/a')
view.click()

## Active students (JMI)

In [13]:
# grab canceled students
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [14]:
active_url = []
active_name = []
for x in soup.find(id = 'js-students-active').find_all('a'):
    print(x['href'] + ' - ' + x.text)
    active_url.append(x['href'])
    active_name.append(x.text)
    

/staff/students/deannalinn - a a
/staff/students/a-glazer - A Glazer
/staff/students/akizzles - A K
/staff/students/osrsnub - Aaron Cherniavsky
/staff/students/aaronc1546 - Aaron Christensen
/staff/students/afeinberg22 - Aaron Feinberg
/staff/students/aamax67 - Aaron Lieberman
/staff/students/aparkening - Aaron Parkening
/staff/students/codingsensei19 - Aaron Wilson
/staff/students/abogailo - Abagail Hudson
/staff/students/abadi230 - Abdullah Bajaman
/staff/students/asafi1997 - Abdul Tawab Ajmal  Safi
/staff/students/4227 - Abel Birhanu
/staff/students/4711 - Abigail Kimbler
/staff/students/softwaree40 - Abiodun   [Wale] Ajagunna
/staff/students/ag212 - Abraham Galva
/staff/students/jdiaz28 - Abraham Moreno
/staff/students/coachcooz - Acusio Bivona
/staff/students/adamcbassett - Adam Bassett
/staff/students/adchang91 - Adam Chang
/staff/students/amcooper - Adam Cooper
/staff/students/adogthewarrior - Adam Fudge
/staff/students/mathycoder - Adam S
/staff/students/aschonely83 - Adam Scho

### Compare Active vs Roster

In [15]:
df_roster = pd.read_csv('cohort_rosters/online-ds-ft_pt-100719 planing - Cohort Roster.csv')
df_roster['full_names']  = df_roster['First Name'] + ' ' +df_roster['Last Name']
df_roster

,Cohort,First Name,Last Name,SlackName,Zoom Name,Email,Enrollment Status,M1Proj Pass,Mod 3 Proj Pass,Msg Re: Mod1 Review 12-11-19,Unnamed: 10,11/19/19 Check -Instruct App,11/19/19 Check-Learn,ACTION NEEDED,full_names
0,FT,Ice,Asortse,Ice,Ice,dantarshi@gmail.com,Day 1 Start,Y,C,NaN,NaN,Y,Y,NaN,Ice Asortse
1,FT,Devin,Belden,Devin B,NaN,devin.belden.28@gmail.com,Day 1 Start,Y,N,NaN,NaN,Y,Y,NaN,Devin Belden
2,FT,Melvin,Gonzalez,"Melvin Gonzalez, Jr.",NaN,mgonzalez713@gmail.com,Day 1 Start,Y,N,NaN,NaN,Y,Y,NaN,Melvin Gonzalez
3,FT,Ru,Kein,Ru,Ru,rukeine@gmail.com,Day 1 Start,Y,TBC,NaN,NaN,Y,Y,NaN,Ru Kein
4,FT,John,Kline,John Kline,John,johnmkline@gmail.com,Day 1 Start,Y,1,NaN,NaN,Y,Y,NaN,John Kline
5,FT,John,Lemaster,John Lemaster,Lemaster,heylemaster@gmail.com,IN - Transfer,Y,1,NaN,NaN,Y,Y,NaN,John Lemaster
6,FT,Rebecca,Mih,Rebecca,NaN,rebecca.mih@gmail.com,IN - Transfer,Y,TBC,NaN,NaN,N,Y,Add to instruct,Rebecca Mih
7,FT,Terry,Ollila,Terry Ollila,NaN,terryollila@gmail.com,Day 1 Start,Y,Y,NaN,NaN,Y,Y,NaN,Terry Ollila
8,FT,Douglas,Steen,Doug Steen,NaN,douglaspsteen@gmail.com,Day 1 Start,Y,Y,NaN,NaN,Y,Y,NaN,Douglas Steen
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_roster.set_index('full_names',inplace=True)

In [17]:
try:
    students_to_drop = ['Anila Qureshi','Ameer Saleemi','Jean Robert Malbranche','Ayaan Harper-Shaw']
    df_roster.drop(students_to_drop,inplace=True)
except:
    print('Could not drop stuents.')

df_roster

,Cohort,First Name,Last Name,SlackName,Zoom Name,Email,Enrollment Status,M1Proj Pass,Mod 3 Proj Pass,Msg Re: Mod1 Review 12-11-19,Unnamed: 10,11/19/19 Check -Instruct App,11/19/19 Check-Learn,ACTION NEEDED
full_names,,,,,,,,,,,,,,
Ice Asortse,FT,Ice,Asortse,Ice,Ice,dantarshi@gmail.com,Day 1 Start,Y,C,NaN,NaN,Y,Y,NaN
Devin Belden,FT,Devin,Belden,Devin B,NaN,devin.belden.28@gmail.com,Day 1 Start,Y,N,NaN,NaN,Y,Y,NaN
Melvin Gonzalez,FT,Melvin,Gonzalez,"Melvin Gonzalez, Jr.",NaN,mgonzalez713@gmail.com,Day 1 Start,Y,N,NaN,NaN,Y,Y,NaN
Ru Kein,FT,Ru,Kein,Ru,Ru,rukeine@gmail.com,Day 1 Start,Y,TBC,NaN,NaN,Y,Y,NaN
John Kline,FT,John,Kline,John Kline,John,johnmkline@gmail.com,Day 1 Start,Y,1,NaN,NaN,Y,Y,NaN
John Lemaster,FT,John,Lemaster,John Lemaster,Lemaster,heylemaster@gmail.com,IN - Transfer,Y,1,NaN,NaN,Y,Y,NaN
Rebecca Mih,FT,Rebecca,Mih,Rebecca,NaN,rebecca.mih@gmail.com,IN - Transfer,Y,TBC,NaN,NaN,N,Y,Add to instruct
Terry Ollila,FT,Terry,Ollila,Terry Ollila,NaN,terryollila@gmail.com,Day 1 Start,Y,Y,NaN,NaN,Y,Y,NaN
Douglas Steen,FT,Douglas,Steen,Doug Steen,NaN,douglaspsteen@gmail.com,Day 1 Start,Y,Y,NaN,NaN,Y,Y,NaN


In [18]:
##
active_students = pd.DataFrame({'name': active_name,'url':active_url})
active_students

,name,url
0,a a,/staff/students/deannalinn
1,A Glazer,/staff/students/a-glazer
2,A K,/staff/students/akizzles
3,Aaron Cherniavsky,/staff/students/osrsnub
4,Aaron Christensen,/staff/students/aaronc1546
...,...,...
2362,aweekes1101@gmail.com,/staff/students/5191
2363,tasmith1990@gmail.com,/staff/students/theheadty
2364,aimeeyang0110@gmail.com,/staff/students/4034
2365,omar.benjamin377@gmail.com,/staff/students/hotxshotx


#### Notes:
- Name contains email addresses for the later students. 
- Not sure if its an error or related to actual missing data

In [19]:
my_students = df_roster.index.to_series()
my_students

full_names
Ice Asortse                    Ice Asortse
Devin Belden                  Devin Belden
Melvin Gonzalez            Melvin Gonzalez
Ru Kein                            Ru Kein
John Kline                      John Kline
John Lemaster                John Lemaster
Rebecca Mih                    Rebecca Mih
Terry Ollila                  Terry Ollila
Douglas Steen                Douglas Steen
NaN                                    NaN
alex beat                        alex beat
Acusio Bivona                Acusio Bivona
Michael Burak                Michael Burak
Darius Fuller                Darius Fuller
Susanna Han                    Susanna Han
Stephen Newman              Stephen Newman
John Ofrecio                  John Ofrecio
Andrea Osika                  Andrea Osika
Jamaal Smith                  Jamaal Smith
Susanna Mir-Syassen    Susanna Mir-Syassen
David Torres                  David Torres
Gerardo Hernandez        Gerardo Hernandez
NaN                                    NaN


In [20]:
active_students['my_student'] = active_students['name'].isin(my_students.values)
active_students

,name,url,my_student
0,a a,/staff/students/deannalinn,False
1,A Glazer,/staff/students/a-glazer,False
2,A K,/staff/students/akizzles,False
3,Aaron Cherniavsky,/staff/students/osrsnub,False
4,Aaron Christensen,/staff/students/aaronc1546,False
...,...,...,...
2362,aweekes1101@gmail.com,/staff/students/5191,False
2363,tasmith1990@gmail.com,/staff/students/theheadty,False
2364,aimeeyang0110@gmail.com,/staff/students/4034,False
2365,omar.benjamin377@gmail.com,/staff/students/hotxshotx,False


In [21]:
df_my_students = active_students.loc[active_students['my_student']==True].copy()
df_my_students

,name,url,my_student
17,Acusio Bivona,/staff/students/coachcooz,True
170,Andrea Osika,/staff/students/andiosika,True
621,Darius Fuller,/staff/students/dfuller22,True
676,Devin Belden,/staff/students/devinbelden,True
699,Douglas Steen,/staff/students/douglaspsteen,True
924,Ice Asortse,/staff/students/dantarshi,True
975,Jamaal Smith,/staff/students/jdsmithwes,True
1109,John Kline,/staff/students/katahdin28,True
1111,John Lemaster,/staff/students/lemaster1,True
1116,John Ofrecio,/staff/students/jofrecio,True


### SAVING LIST OF MY STUDENTS TO CSV

In [25]:
# df_my_students.to_csv('jmi_students_from_active_students.csv')
df_my_students = pd.read_csv('jmi_students_from_active_students.csv',index_col=0)
df_my_students

,name,url,my_student
17,Acusio Bivona,/staff/students/coachcooz,True
170,Andrea Osika,/staff/students/andiosika,True
621,Darius Fuller,/staff/students/dfuller22,True
676,Devin Belden,/staff/students/devinbelden,True
699,Douglas Steen,/staff/students/douglaspsteen,True
924,Ice Asortse,/staff/students/dantarshi,True
975,Jamaal Smith,/staff/students/jdsmithwes,True
1109,John Kline,/staff/students/katahdin28,True
1111,John Lemaster,/staff/students/lemaster1,True
1116,John Ofrecio,/staff/students/jofrecio,True


### Further Processing My Students

In [26]:
df = df_my_students.copy()

In [27]:
def make_instructor_learn_url(partial_url):
    """Prepends the base url to relative url for df.apply"""
    base_url = 'https://instruction.learn.co' + partial_url
    return base_url

df['full_url'] = df['url'].apply(make_instructor_learn_url)
df

,name,url,my_student,full_url
17,Acusio Bivona,/staff/students/coachcooz,True,https://instruction.learn.co/staff/students/co...
170,Andrea Osika,/staff/students/andiosika,True,https://instruction.learn.co/staff/students/an...
621,Darius Fuller,/staff/students/dfuller22,True,https://instruction.learn.co/staff/students/df...
676,Devin Belden,/staff/students/devinbelden,True,https://instruction.learn.co/staff/students/de...
699,Douglas Steen,/staff/students/douglaspsteen,True,https://instruction.learn.co/staff/students/do...
924,Ice Asortse,/staff/students/dantarshi,True,https://instruction.learn.co/staff/students/da...
975,Jamaal Smith,/staff/students/jdsmithwes,True,https://instruction.learn.co/staff/students/jd...
1109,John Kline,/staff/students/katahdin28,True,https://instruction.learn.co/staff/students/ka...
1111,John Lemaster,/staff/students/lemaster1,True,https://instruction.learn.co/staff/students/le...
1116,John Ofrecio,/staff/students/jofrecio,True,https://instruction.learn.co/staff/students/jo...


### Now grab info from each student's learn page

In [ ]:
# my_html = driver.page_source
# soup = BeautifulSoup(my_html, 'html.parser')

In [28]:
# ## Example edge case of not having a maximum num of sessions
# coach_max_link = 'https://instruction.learn.co/staff/students/abbyox'
# driver.get(coach_max_link)

In [29]:
test_link = df['full_url'].iloc[1]
driver.get(test_link)

#### Divergence

In [38]:
def get_student_info_from_full_url(full_url):
    """Get the `uk-card-body` text from student's instruction.learn full url"""
    driver.get(test_link)
    my_html = driver.page_source
    student_soup = BeautifulSoup(my_html, 'html.parser')
    tag = student_soup.find('div',attrs={'class':'uk-card-body'})
    student_info = tag#.text
    print('Function used to return the .text, now it returns the tag itself.')
    return student_info


In [70]:
test_link
tags = get_student_instruct_cards(test_link)

In [72]:
tags[0].text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlast active 12/22/19\nSection: Data Science\n\nEducational Coach: Currently assigned to Dara Paoletti\n\nEducational Coach Status Changes?\n\nCareer Coach: Not currently assigned\n Cohort Lead: \n\n          220 completed lessons (31%, 0 questions asked)\nJoined Learn 10/07/19 (3 months ago)\nStarted FSWD 10/07/19 (3 months ago)\n'

In [81]:
import re
def get_student_dict_from_student_info(student_info_tag):
    """Uses regex to parse info from student's
    instruction.learn.co/staff/students/____ page:
    

    Ex Use:
    full_link = df['full_url'].iloc[i]
    student_info_tag =  get_student_info_from_full_url(full_link)
    
    student_dict = get_student_dict_from_student_info(student_info_tag)
    """
    student_info = student_info_tag.text

    import re

    student_dict = {}

    ## Cohort Lead
    re_cohort_lead = re.compile(r"Cohort Lead: (\w* \w*)")
    student_dict['cohort_lead'] = re_cohort_lead.findall(student_info)#[0]

    ## Ed Coach
    re_ed_coach = re.compile(r"Educational Coach: Currently assigned to (\w*.\w*)")
    student_dict['ed_coach'] = re_ed_coach.findall(student_info)#[0]


    ## Last Active
    re_last_active = re.compile(r"last.active.(\d{2,}/\d{2,}/\d{2,})")
    student_dict['last_active'] = re_last_active.findall(student_info)#[0]


    ## Completed Lessons
    re_comp_lessons = re.compile(r"(\d{1,4}).completed lessons")
    student_dict['completed_lessons'] = re_comp_lessons.findall(student_info)#[0]

    ## Joined learn
    re_joined = re.compile(r"Joined Learn (\d{2}/\d{2}/\d{2})")
    student_dict['joined'] = re_joined.findall(student_info)#[0]



    ## Career Coach - MUST BETTER ADDRESS NOT CURRENTLY ASSIGNED
    re_career_coach= re.compile(r"Career Coach: (\w* \w*)")
    student_dict['career_coach'] = re_career_coach.findall(student_info)#[0]

    re_section = re.compile(r"Section: ([A-Za-z\t .]+)")
    student_dict['section'] = re_section.findall(student_info)#[0]
    
    ## Adding Unpacking of lists via dict loop
    
    for k,v in student_dict.items():
        try:
            student_dict[k] = v[0]
        except:
            continue
        
    return student_dict


def get_program_info(program_info_tag):
    
    program_info = program_info_tag.text
    
    program_dict = {}
    
    re_coach_sess= re.compile(r"Touchpoints: (\d{1,2}) (of (\d{1,2}))?") #"Touchpoints: (\d{1,2}) of (\d{1,2})")
    num_ed_coach_sess = re_coach_sess.findall(program_info)
    
    num_used= num_ed_coach_sess[0][0]
    num_avail = num_ed_coach_sess[0][-1]
    
    program_dict['ed_coach_avail'] = num_avail
    program_dict['ed_coach_used'] = num_used

    
    re_cohort = re.compile(r"Cohort: ([A-Za-z-\d]+)")
    program_dict['cohort'] = re_cohort.findall(program_info)[0]
    
    re_pacing = re.compile(r"Pacing: ([A-Za-z-\d ]+)")
    program_dict['pacing'] =re_pacing.findall(program_info)[0]
    
    
    return program_dict
    

def get_profile_info(profile_info_tag):
    
    profile_dict = {}
    profile_text = profile_info_tag.text
    grad_date = re.compile(r"Ideal Graduation Date: (([\d-]?)*)")
    
    profile_dict['grad_date'] = grad_date.findall(profile_text)#[0][0]
    
    
    re_comm_level = re.compile(r"Commitment Level: (\w* Time)")
    profile_dict['commitment_level'] = re_comm_level.findall(profile_text)#[0]
    
    re_lives_in = re.compile(r"Lives in ([A-Za-z ,]+)")
    profile_dict['lives_in'] = re_lives_in.findall(profile_text)#[0]
    return profile_dict

In [82]:
"""tags = get_student_instruct_cards(full_url)
student_data_dict =  process_student_instruct_cards(tags)"""

def get_student_instruct_cards(full_url):
    """Get the `uk-card-body` text from student's instruction.learn full url"""
    driver.get(test_link)
    my_html = driver.page_source
    student_soup = BeautifulSoup(my_html, 'html.parser')
    tags = student_soup.find_all('div',attrs={'class':'uk-card-body'})
    return tags

def process_student_instruct_cards(tags_with_cards):
    """Calls on get_student_dict_from_student_info,
    get_program_info, and get_profile_info to make
    student_data_dict.
    """
    student_info_dict = {}
    ### Tags have all 3 boxes
    # blue student_info 
    student_info = tags[0]
    student_info_dict['student_info'] = get_student_dict_from_student_info(student_info)

    # red student_info 
    program_info = tags[1]
    student_info_dict['program_info'] = get_program_info(program_info)

    # purple profile 
    profile_info = tags[2]
    
    student_info_dict['profile_info'] = get_profile_info(profile_info)
    
    return student_info_dict 
    

In [85]:
## Using the streamlined functions
test_student_tags = get_student_instruct_cards(test_link)
test_student_dict = process_student_instruct_cards(test_student_tags)
test_student_dict

{'student_info': {'cohort_lead': [],
  'ed_coach': 'Dara Paoletti',
  'last_active': '12/22/19',
  'completed_lessons': '220',
  'joined': '10/07/19',
  'career_coach': 'Not currently',
  'section': 'Data Science'},
 'program_info': {'ed_coach_avail': '11',
  'ed_coach_used': '2',
  'cohort': 'online-ds-pt-100719',
  'pacing': 'Part Time'},
 'profile_info': {'grad_date': [], 'commitment_level': [], 'lives_in': []}}

In [86]:
## TROUBLESHOOTING
tags = get_student_instruct_cards(test_link)
student_info_tag = tags[0]
student_info = student_info_tag.text

student_info

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlast active 12/22/19\nSection: Data Science\n\nEducational Coach: Currently assigned to Dara Paoletti\n\nEducational Coach Status Changes?\n\nCareer Coach: Not currently assigned\n Cohort Lead: \n\n          220 completed lessons (31%, 0 questions asked)\nJoined Learn 10/07/19 (3 months ago)\nStarted FSWD 10/07/19 (3 months ago)\n'

# BOOKMARK 12/23/19 5:45pm
- Andi doesn't officially have me listed as her cohort lead and caused an error when triyng to get info

- Replaced unpacking of tags from the end of each line to a for loop with at try except clause

In [80]:
# student_info = student_info_tag.text

import re

student_dict = {}

## Cohort Lead
re_cohort_lead = re.compile(r"Cohort Lead: (\w* \w*)")
student_dict['cohort_lead'] = re_cohort_lead.findall(student_info)[0]
student_dict

IndexError: list index out of range

In [69]:
test_student_dict

{'student_info': {'cohort_lead': [],
  'ed_coach': ['Dara Paoletti'],
  'last_active': ['12/22/19'],
  'completed_lessons': ['220'],
  'joined': ['10/07/19'],
  'career_coach': ['Not currently'],
  'section': ['Data Science']},
 'program_info': {'ed_coach_avail': '11',
  'ed_coach_used': '2',
  'cohort': 'online-ds-pt-100719',
  'pacing': 'Part Time'},
 'profile_info': {'grad_date': [], 'commitment_level': [], 'lives_in': []}}

In [ ]:
get_student_info_from_full_url()

In [ ]:
my_html = driver.page_source
test_soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# test_soup

In [ ]:
tags = test_soup.findAll('div',attrs={'class':'uk-card-body'})
# tags

### Making 3 functions (one per info box)

In [ ]:
### Tags have all 3 boxes
# blue student_info 
student_info = tags[0]#.text

# red student_info 
program_info = tags[1]

# purple profile 
profile_info = tags[2]

# 📓BOOKMARK: JAMES 

#### Scraping links from 'custom-badges'

In [ ]:
badges = student_info.find_all('div',attrs={'class':'custom-badges'})
links = badges[0].find_all(href=True)

In [ ]:
## List of Links 
slack_url = links[0]['href']
learn_url = links[1]['href']
github_url = links[2]['href']
mailto_url = links[3]['href']
projects_url = links[4]['href'] 
grad_dash_url = links[5]['href']

In [ ]:
SLACK = links[0]
SLACK

In [ ]:
links = badges[0].find_all(href=True)
# links[0].find_all('href')
len(links)

In [ ]:
learn_link = soup.find('div', class_ = 'custom-badges').findAll('a', class_ = 'custom-badge-icon')[1]['href']


In [ ]:
tag = links[0]
tag.find_all('a')

In [ ]:
student_info.find_all(#`attrs={'class':'custom-bages'})

In [44]:
# def get_program_info(program_info):
    
#     program_dict = {}
    
#     re_coach_sess= re.compile(r"Touchpoints: (\d{1,2}) (of (\d{1,2}))?") #"Touchpoints: (\d{1,2}) of (\d{1,2})")
#     num_ed_coach_sess = re_coach_sess.findall(program_info.text)
    
#     num_used= num_ed_coach_sess[0][0]
#     num_avail = num_ed_coach_sess[0][-1]
    
#     program_dict['ed_coach_avail'] = num_avail
#     program_dict['ed_coach_used'] = num_used

    
#     re_cohort = re.compile(r"Cohort: ([A-Za-z-\d]+)")
#     program_dict['cohort'] = re_cohort.findall(program_info.text)[0]
    
#     re_pacing = re.compile(r"Pacing: ([A-Za-z-\d ]+)")
#     program_dict['pacing'] =re_pacing.findall(program_info.text)[0]
    
    
#     return program_dict
    

# def get_profile_info(profile_info):
    
#     profile_dict = {}
#     profile_text = profile_info.text
#     grad_date = re.compile(r"Ideal Graduation Date: (([\d-]?)*)")
    
#     profile_dict['grad_date'] = grad_date.findall(profile_text)[0][0]
    
    
#     re_comm_level = re.compile(r"Commitment Level: (\w* Time)")
#     profile_dict['commitment_level'] = re_comm_level.findall(profile_text)[0]
    
#     re_lives_in = re.compile(r"Lives in ([A-Za-z ,]+)")
#     profile_dict['lives_in'] = re_lives_in.findall(profile_text)[0]
#     return profile_dict
    
     

In [ ]:
get_program_info(program_info)

### getting profile info
- Note: Current progress satisfication is difficult to grab (profile info doesn't have new lines)
    - Skip for now

In [45]:
# def get_profile_info(profile_info):
    
#     profile_dict = {}
#     profile_text = profile_info.text
#     grad_date = re.compile(r"Ideal Graduation Date: (([\d-]?)*)")
    
#     profile_dict['grad_date'] = grad_date.findall(profile_text)[0][0]
    
    
#     re_comm_level = re.compile(r"Commitment Level: (\w* Time)")
#     profile_dict['commitment_level'] = re_comm_level.findall(profile_text)[0]
    
#     re_lives_in = re.compile(r"Lives in ([A-Za-z ,]+)")
#     profile_dict['lives_in'] = re_lives_in.findall(profile_text)[0]
#     return profile_dict
    
     

In [ ]:
get_profile_info(profile_info)

In [ ]:
profile_info.text

In [ ]:
profile_info

In [ ]:
# help(profile_info)

In [ ]:
profile_info.find_all('p')

In [ ]:
h5s = profile_info.find_all('h5')#,text='Onboarding')#,text="Type:"
onboarding_info = h5s[-1]
onboarding_info.

In [ ]:
program_info.text

In [ ]:
get_student_dict_from_student_info(student_info.text)

In [ ]:
student_info.text

In [ ]:
import re
re_coach_sess= re.compile(r"Touchpoints: (?P<avial>\d{1,2}) (of (?P<max>\d{1,2}))?") #"Touchpoints: (\d{1,2}) of (\d{1,2})")
num_ed_coach_sess = re_coach_sess.findall(program_info.text)
num_used= num_ed_coach_sess[0][0]
num_avail = num_ed_coach_sess[0][-1]
num_used,num_avail


In [ ]:
re_coach_sess.findall(student_info.text)

In [ ]:
# import pandas as pd
# df_pt = pd.read_csv("/Users/jamesirving/.secret/pt_cohort_data.csv")
# df_ft = pd.read_csv("/Users/jamesirving/.secret/ft_cohort_data.csv")

# display(df_pt.head())
# display(df_ft.head())

In [ ]:
# tags[3].text

### Alternative Since can't get program url via Jeff's ` 'uk-card-title uk-margin-remove-bottom'`

In [ ]:
tag = test_soup.find('div',attrs={'class':'uk-card-body'})
student_info = tag.text
student_info


### Using regex to capture fields from student_info


### Example usage of James' functions

In [ ]:
test_url = df['full_url'].iloc[0]
student_info = get_student_info_from_full_url(test_url)
student_dict = get_student_dict_from_student_info(student_info)
student_dict

## 📓BOOKMARK FOR JAMES & JEFF
- Understanding what info Jeff's functions grabs vs info that my functions grab and how to marry the 2

In [ ]:
## JEFF's FUCNTION'
def get_info_full_url(full_url):
#     driver.get('https://instruction.learn.co' + url_part2)
    driver.get(full_url)
    time.sleep(2)
    
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    
    ## Targeting the Red box on instruct.learn profile
    program = soup.findAll('h3', class_ = 'uk-card-title uk-margin-remove-bottom')[1].text.strip()

    ## Searched for text in div's with uk-card-body class
    start_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Educational')
    stop_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Maximum')
    my_str = soup.findAll('div', class_ = 'uk-card-body')[1].text[start_ed_coach:stop_ed_coach]
    ed_coach_taken = [int(s) for s in my_str.split() if s.isdigit()][0]
    try:
        ed_coach_avail = [int(s) for s in my_str.split() if s.isdigit()][1]
    except IndexError:
        ed_coach_avail = ''
    cohort = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[0].text
    pacing = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[1].text
    try:
        past_cohorts = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[2].text
    except IndexError:
        past_cohorts = ''
    learn_link = soup.find('div', class_ = 'custom-badges').findAll('a', class_ = 'custom-badge-icon')[1]['href']
    
    ##James's return
    student_dict = dict( ed_coach_taken=ed_coach_taken,
                        ed_coach_avail = ed_coach_avail,
                        cohort = cohort,
                        pacing=pacing,
                        past_cohorts=past_cohorts,
                        learn_link=learn_link)
    return student_dict
    ### Jeff's Return:
#     return program, ed_coach_taken, ed_coach_avail, cohort, pacing, past_cohorts, learn_link

test_dict =  get_info_full_url(test_link)

In [ ]:
test_dict

### Got up to Scraping Program from student's instruct page

In [ ]:
# test_soup.find('a')

In [ ]:
# res = test_soup.findall('a')

# Jeff's Original Code:

## Cancelled students (JH)

In [ ]:
# # cancelled students
# canceled = driver.find_element_by_xpath('//*[@id="student-index-card"]/h2[2]')
# canceled.click()

In [ ]:
# # grab canceled students
# my_html = driver.page_source
# soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# canceled_url = []
# canceled_name = []
# for x in soup.find(id = 'js-students-cancelled').find_all('a'):
#     print(x['href'] + ' - ' + x.text)
#     canceled_url.append(x['href'])
#     canceled_name.append(x.text)

In [ ]:
# canceled_df = pd.DataFrame({'name': canceled_name,
#                             'url': canceled_url})

In [ ]:
# canceled_df.head()

## Graduated Students

In [ ]:
# soup.find(id = 'js-students-graduated').find_all('a')

In [ ]:
# graduated_url = []
# graduated_name = []
# for x in soup.find(id = 'js-students-graduated').find_all('a'):
#     print(x['href'] + ' - ' + x.text)
#     graduated_url.append(x['href'])
#     graduated_name.append(x.text)

In [ ]:
# graduated_df = pd.DataFrame({'name': graduated_name,
#                             'url': graduated_url})

In [ ]:
# graduated_df.head()

## Further Processing Cancelled Students

In [ ]:
canceled_df.head()

In [ ]:
canceled_df.loc[0, 'url']

In [ ]:
https://instruction.learn.co/staff/students/siguy12

### Making absolute URLS

In [ ]:
'https://instructor.learn.co' + canceled_df.loc[0, 'url']

In [ ]:
driver.get('https://instruction.learn.co/' + canceled_df.loc[0, 'url'])

### grab program, pacing, past_cohorts, learn.co url

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# program
soup.findAll('h3', class_ = 'uk-card-title uk-margin-remove-bottom')[1].text.strip()

In [ ]:
# ed coach check ins 
soup.findAll('div', class_ = 'uk-card-body')[1]

In [ ]:
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')

In [ ]:
start_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Educational')

In [ ]:
stop_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Maximum')

In [ ]:
my_str = soup.findAll('div', class_ = 'uk-card-body')[1].text[start_ed_coach:stop_ed_coach]

In [ ]:
my_str

In [ ]:
[int(s) for s in my_str.split() if s.isdigit()]

In [ ]:
# cohort 
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[0].text

In [ ]:
# pacing
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[1].text

In [ ]:
# past cohorts 
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[2].text

In [ ]:
# learn.co link
soup.find('div', class_ = 'custom-badges').findAll('a', class_ = 'custom-badge-icon')[1]['href']

In [ ]:
driver.get('http://learn.co/siguy12')

In [ ]:
def get_info(url_part2):
    driver.get('https://instruction.learn.co' + url_part2)
    time.sleep(2)
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    program = soup.findAll('h3', class_ = 'uk-card-title uk-margin-remove-bottom')[1].text.strip()
    start_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Educational')
    stop_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Maximum')
    my_str = soup.findAll('div', class_ = 'uk-card-body')[1].text[start_ed_coach:stop_ed_coach]
    ed_coach_taken = [int(s) for s in my_str.split() if s.isdigit()][0]
    try:
        ed_coach_avail = [int(s) for s in my_str.split() if s.isdigit()][1]
    except IndexError:
        ed_coach_avail = ''
    cohort = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[0].text
    pacing = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[1].text
    try:
        past_cohorts = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[2].text
    except IndexError:
        past_cohorts = ''
    learn_link = soup.find('div', class_ = 'custom-badges').findAll('a', class_ = 'custom-badge-icon')[1]['href']
    return program, ed_coach_taken, ed_coach_avail, cohort, pacing, past_cohorts, learn_link

In [ ]:
get_info('/staff/students/siguy12')

In [ ]:
get_info('/staff/students/perafolio')

In [ ]:
canceled_df.head(2)

In [ ]:
prog_list = []
ed_coach_used_list = []
ed_coach_avail_list = []
cohort_list = []
pacing_list = []
past_cohorts_list = []
learn_list = []
for index, x in enumerate(canceled_df['url']):
    prog_elem, ed_elem1, ed_elem2, cohort_elem, pacing_elem, past_elem, learn_elem = get_info(x)
    prog_list.append(prog_elem)
    ed_coach_avail_list.append(ed_elem1)
    ed_coach_used_list.append(ed_elem1)
    cohort_list.append(cohort_elem)
    pacing_list.append(pacing_elem)
    past_cohorts_list.append(past_elem)
    learn_list.append(learn_elem)
    if index % 50 == 0:
        print(f'Completed {index} out of {len(canceled_df["url"])}')

In [ ]:
canceled_df['program'] = prog_list
canceled_df['cohort'] = cohort_list
canceled_df['pacing'] = pacing_list
canceled_df['past_cohorts'] = past_cohorts_list
canceled_df['ed_coach_used'] = ed_coach_used_list
canceled_df['ed_coach_available'] = ed_coach_avail_list
canceled_df['learn_url'] = learn_list

In [ ]:
canceled_df.to_csv('canceled_df.csv', index = False)

In [ ]:
prog_list = []
ed_coach_used_list = []
ed_coach_avail_list = []
cohort_list = []
pacing_list = []
past_cohorts_list = []
learn_list = []
for index, x in enumerate(graduated_df['url']):
    prog_elem, ed_elem1, ed_elem2, cohort_elem, pacing_elem, past_elem, learn_elem = get_info(x)
    prog_list.append(prog_elem)
    ed_coach_avail_list.append(ed_elem1)
    ed_coach_used_list.append(ed_elem1)
    cohort_list.append(cohort_elem)
    pacing_list.append(pacing_elem)
    past_cohorts_list.append(past_elem)
    learn_list.append(learn_elem)
    if index % 50 == 0:
        print(f'Completed {index} out of {len(graduated_df["url"])}')

In [ ]:
graduated_df['program'] = prog_list
graduated_df['cohort'] = cohort_list
graduated_df['pacing'] = pacing_list
graduated_df['past_cohorts'] = past_cohorts_list
graduated_df['ed_coach_used'] = ed_coach_used_list
graduated_df['ed_coach_available'] = ed_coach_avail_list
graduated_df['learn_url'] = learn_list

In [ ]:
graduated_df.to_csv('graduated_df.csv', index = False)

In [ ]:
graduated_df.head()

In [ ]:
graduated_df[graduated_df['name'] == 'Jon Tyler']

In [ ]:
graduated_df['program'].value_counts()

In [ ]:
canceled_df['program'].value_counts()

In [ ]:
56 / 129

In [ ]:
131 / 321

In [ ]:
canceled_df[canceled_df['program'].str.contains('Data Science')]['pacing'].value_counts()

In [ ]:
graduated_df[graduated_df['program'].str.contains('Data Science')]['pacing'].value_counts()

In [ ]:
# data for canceled students 
cancled_sp = canceled_df[(canceled_df['program'].str.contains('Data Science')) & 
           (canceled_df['pacing'].str.contains('Self Paced')) & 
           (canceled_df['past_cohorts'] == '')]

In [ ]:
cancled_sp

In [ ]:
driver.close()

In [ ]:
driver = webdriver.Chrome()

In [ ]:
test_url = cancled_sp.loc[18, 'learn_url']

In [ ]:
driver.get(test_url)

In [ ]:
sign_in_gh = driver.find_element_by_xpath('//*[@id="js--region-main"]/div/div[1]/div/div/div[5]/form/div/div[6]/a')

In [ ]:
sign_in_gh.click()

## Username for Github

In [ ]:
un = driver.find_element_by_xpath('//*[@id="login_field"]')
un.send_keys()

## Password for Github

In [ ]:
pw = driver.find_element_by_xpath('//*[@id="password"]')
pw.send_keys()

In [ ]:
sn = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sn.click()

In [ ]:
time.sleep(4)

In [ ]:
driver.get(test_url)

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
soup.find('h4', class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text

In [ ]:
while True:
    try:
        lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
        lm.click()
        time.sleep(2)
    except:
        break

In [ ]:
driver.get('http://learn.co/govenfist')

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# curriculum
soup.find('h4', class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text

In [ ]:
#lessons completed
soup.find('span', class_ = 'heading heading--level-1 heading--color-green heading--font-size-largest heading--weight-lighter').text

In [ ]:
# last lab 
soup.find('h4', class_ = 'heading heading--level-1 heading--color-blue heading--weight-lighter').text

In [ ]:
while True:
    try:
        lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
        lm.click()
        time.sleep(2)
    except:
        break

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
first_date = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')[0]

In [ ]:
# date
first_date.find('div', class_ = 'heading heading--level-2 heading--color-grey-light').text

In [ ]:
lessons = first_date.find('ul',
                          class_ = 'list list--spacing-large list--separators-grey-faintest')

In [ ]:
# lessons
for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
    print(l_lesson.find('a').text)

In [ ]:
# iterate through the entire 
all_labs = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')

dates_list = []
lessons_list = []

for d in all_labs:
    date = d.find('div', 
                  class_ = 'heading heading--level-2 heading--color-grey-light').text
    lessons = d.find('ul',
                          class_ = 'list list--spacing-large list--separators-grey-faintest')
    for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
        dates_list.append(date)
        lessons_list.append(l_lesson.find('a').text)
        print(l_lesson.find('a').text)

In [ ]:
len(dates_list)

In [ ]:
len(lessons_list)

In [ ]:
df_lessons = pd.DataFrame({'dates': dates_list,
                          'lesson': lessons_list})

In [ ]:
df_lessons.tail(15)

In [ ]:
cancled_sp.loc[18, 'url'].split('/')[-1]

In [ ]:
cancled_sp.head()

In [ ]:
cancled_sp['curriculum'] = ''
cancled_sp['lessons_completed'] = 0
cancled_sp['last_lesson'] = ''

In [ ]:
cancled_sp

In [ ]:
#driver.close()
driver = webdriver.Chrome()

driver.get('http://learn.co/superguy200')
    
# sign into github 

# click sign up with github button
sign_in_gh = driver.find_element_by_xpath('//*[@id="js--region-main"]/div/div[1]/div/div/div[5]/form/div/div[6]/a')
sign_in_gh.click()

# enter github username 
un = driver.find_element_by_xpath('//*[@id="login_field"]')
un.send_keys()

# enter github password
pw = driver.find_element_by_xpath('//*[@id="password"]')
pw.send_keys()

# click submit
sn = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sn.click()

for index, row in cancled_sp.iterrows():
    
    # visit url
    driver.get(row['learn_url'])
    
    # let load
    time.sleep(2)
    
    # grab html
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    
    # check to see if they are in bootcamp prep
    curriculum = soup.find('h4', 
              class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text
    print(f'{row["name"]} - {curriculum}')
    if curriculum == 'Bootcamp Prep':
        cancled_sp.loc[index, 'curriculum'] = curriculum
        continue
    else:
        # add curriculum
        cancled_sp.loc[index, 'curriculum'] = curriculum
        
        # grab lessons completed 
        les_completed = soup.find('span', 
                                   class_ = 'heading heading--level-1 heading--color-green heading--font-size-largest heading--weight-lighter').text
        cancled_sp.loc[index, 'lessons_completed'] = les_completed
        
        # grab last lesson completed 
        las_completed = soup.find('h4', 
                                  class_ = 'heading heading--level-1 heading--color-blue heading--weight-lighter').text
        cancled_sp.loc[index, 'last_lesson'] = las_completed
        
        # click load more a bunch
        while True:
            try:
                lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
                lm.click()
                time.sleep(2)
            except:
                break
                
        # grab html
        time.sleep(2)
        my_html = driver.page_source
        soup = BeautifulSoup(my_html, 'html.parser')
        
        all_labs = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')

        dates_list = []
        lessons_list = []

        for d in all_labs:
            date = d.find('div', 
                          class_ = 'heading heading--level-2 heading--color-grey-light').text
            lessons = d.find('ul',
                                  class_ = 'list list--spacing-large list--separators-grey-faintest')
            for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
                dates_list.append(date)
                lessons_list.append(l_lesson.find('a').text)
                
        df_ls = pd.DataFrame({'dates': dates_list,
                          'lesson': lessons_list})
        
        df_ls.to_csv(f'./lessons/canceled/{row["url"].split("/")[-1]}.csv')

In [ ]:
# graduated students 
grad_sp = graduated_df[(graduated_df['program'].str.contains('Data Science')) & 
            (graduated_df['pacing'].str.contains('Self Pace')) & 
            (graduated_df['name'] != 'Emily Pfeifer')]

In [ ]:
grad_sp['curriculum'] = ''
grad_sp['lessons_completed'] = 0
grad_sp['last_lesson'] = ''

In [ ]:
#driver.close()
driver = webdriver.Chrome()

driver.get('http://learn.co/superguy200')
    
# sign into github 

# click sign up with github button
sign_in_gh = driver.find_element_by_xpath('//*[@id="js--region-main"]/div/div[1]/div/div/div[5]/form/div/div[6]/a')
sign_in_gh.click()

# enter github username 
un = driver.find_element_by_xpath('//*[@id="login_field"]')
un.send_keys()

# enter github password
pw = driver.find_element_by_xpath('//*[@id="password"]')
pw.send_keys()

# click submit
sn = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sn.click()

for index, row in grad_sp.iterrows():
    
    # visit url
    driver.get(row['learn_url'])
    
    # let load
    time.sleep(2)
    
    # grab html
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    
    # check to see if they are in bootcamp prep
    curriculum = soup.find('h4', 
              class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text
    print(f'{row["name"]} - {curriculum}')
    if curriculum == 'Bootcamp Prep':
        grad_sp.loc[index, 'curriculum'] = curriculum
        continue
    else:
        # add curriculum
        grad_sp.loc[index, 'curriculum'] = curriculum
        
        # grab lessons completed 
        les_completed = soup.find('span', 
                                   class_ = 'heading heading--level-1 heading--color-green heading--font-size-largest heading--weight-lighter').text
        grad_sp.loc[index, 'lessons_completed'] = les_completed
        
        # grab last lesson completed 
        las_completed = soup.find('h4', 
                                  class_ = 'heading heading--level-1 heading--color-blue heading--weight-lighter').text
        grad_sp.loc[index, 'last_lesson'] = las_completed
        
        # click load more a bunch
        while True:
            try:
                lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
                lm.click()
                time.sleep(2)
            except:
                break
                
        # grab html
        time.sleep(2)
        my_html = driver.page_source
        soup = BeautifulSoup(my_html, 'html.parser')
        
        all_labs = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')

        dates_list = []
        lessons_list = []

        for d in all_labs:
            date = d.find('div', 
                          class_ = 'heading heading--level-2 heading--color-grey-light').text
            lessons = d.find('ul',
                                  class_ = 'list list--spacing-large list--separators-grey-faintest')
            for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
                dates_list.append(date)
                lessons_list.append(l_lesson.find('a').text)
                
        df_ls = pd.DataFrame({'dates': dates_list,
                          'lesson': lessons_list})
        
        df_ls.to_csv(f'./lessons/grad/{row["url"].split("/")[-1]}.csv')

In [ ]:
cancled_sp.to_csv('canceled_sp.csv', index = False)
grad_sp.to_csv('grad_sp.csv', index = False)

In [ ]:
grad_sp